In [1]:
from csense.benchmark import cqa
# !pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz
# !pip3.7 install -r requirements.txt
# ! python -m spacy download en_core_web_sm
from PyDictionary import PyDictionary
import spacy
import nltk

dictionary=PyDictionary()
# import en_core_web_sm

# nlp = en_core_web_sm.load()
# nlp = spacy.load("/Users/anushkabaoni/Downloads/Cornell Courses-Spring2020/Advanced AI/CommonsenseQADataset/venv/lib/python3.7/site-packages/en_core_web_sm")
# nlp = spacy.load("../CommonsenseQADataset/venv/lib/python3.7/site-packages/en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

In [2]:
dataset = cqa.getDataSet()

In [3]:
what_questions = []
where_questions = []
why_questions = []
who_questions = []
which_questions = []
when_questions = []
how_questions = []

rand = []

for question in dataset:
    assigned = False
    text = set(nltk.word_tokenize(question['question']['stem'].lower()))
    if("what" in text and "where" not in text and "why" not in text and "who" not in text and "when" not in text and "how" not in text and "which" not in text):
        what_questions.append(question)
        continue
    if("what" not in text and "where" in text and "why" not in text and "who" not in text and "when" not in text and "how" not in text and "which" not in text):
        where_questions.append(question)
        continue
    if("what" not in text and "where" not in text and "why" in text and "who" not in text and "when" not in text and "how" not in text and "which" not in text):
        why_questions.append(question)
        continue
    if("what" not in text and "where" not in text and "why" not in text and "who" in text and "when" not in text and "how" not in text and "which" not in text):
        who_questions.append(question)
        continue
    if("what" not in text and "where" not in text and "why" not in text and "who" not in text and "when" in text and "how" not in text and "which" not in text):
        when_questions.append(question)
        continue
    if("what" not in text and "where" not in text and "why" not in text and "who" not in text and "when" not in text and "how" in text and "which" not in text):
        how_questions.append(question)
        continue     
    if("what" not in text and "where" not in text and "why" not in text and "who" not in text and "when" not in text and "how" not in text and "which" in text):
        which_questions.append(question)
        continue        
    # whether, whose, whom, how
    
    doc = nlp(question['question']['stem'].lower())
    for token in doc:
        if(token.dep_ == 'ROOT'):
            childern = [str(child) for child in token.children]         
            if("what" in childern):
                what_questions.append(question)
                assigned=True
                break
            elif("where" in childern):
                where_questions.append(question)
                assigned=True
                break
            elif("why" in childern):
                why_questions.append(question)
                assigned=True
                break
            elif("who" in childern):
                who_questions.append(question)
                assigned=True
                break
            elif("how" in childern):
                how_questions.append(question)
                assigned=True
                break
            elif("which" in childern):
                which_questions.append(question)
                assigned=True
                break
            elif("when" in childern):
                when_questions.append(question)
                assigned=True
                break
    if not assigned:
        rand.append(question)
        for token in doc:
            if("what" == token.text and 'dobj' == token.dep_):
                what_questions.append(question)
                break

print(len(dataset))
print("what : ", len(what_questions))
print("where : ",len(where_questions))
print("why : ",len(why_questions))
print("who : ",len(who_questions))
print("how : ",len(how_questions))
print("which : ",len(which_questions))
print("when : ",len(when_questions))
print(len(what_questions) + len(where_questions) + len(why_questions) + len(who_questions) + len(how_questions) + len(which_questions) + len(when_questions))

9741
what :  5821
where :  2860
why :  241
who :  50
how :  215
which :  44
when :  123
9354


In [74]:
def getParsedSentence(doc):
    subj = []
    verb = []
    obj = []
    root = []
    prop = []

    for token in doc:
        if(token.dep_ == "nsubj"):
            subj.append(token.text)

        elif(token.dep_ == "advcl"):
            verb.append(token.text)
        
        elif(token.dep_ == "amod"):
            prop.append(token.text)

        elif(token.dep_ == "dobj"):
            obj.append(token.text)
        
        elif(token.dep_ == "ROOT"):
            root.append(token.text)
            verb.append(token.text)
        
        print(token.text, token.dep_, token.head.text, token.head.pos_,[child for child in token.children])

    print(subj)
    print(verb)
    print(obj)
    print(root)
    print(prop)


In [115]:
def getSentence(doc):
    phrase = []
    sub = []
    verb = []
    obj = []
    phrases = []

    for chunk in doc.noun_chunks:
        if chunk.root.dep_ == 'nsubj':
            
            # if len(sub) == 1:               # tackle 2 subjects one after the other
            #     phrases.append(phrase)
            #     sub = []
            #     verb = []
            #     obj = []
            #     phrase = []

            # sub = chunk.root.text

            sub =  chunk.text
            verb = chunk.root.head.text
            phrase = []
            phrase.append(sub)
            phrase.append(verb)

        elif chunk.root.dep_ == 'dobj' or chunk.root.dep_ == 'pobj':

            # if len(obj) == 1:
            #     phrase.append(obj)
            #     phrases.append(phrase)
            #     verb = []
            #     obj = []
            #     phrase = phrase[0:1]

            obj = chunk.text

            verb = chunk.root.head.text
            if len(phrase) == 2 and phrase[1] != verb:
                phrase[1] += " " + verb
            elif len(phrase) ==1:
                phrase.append(verb)
            
            phrase.append(obj)

            # phrases.append(phrase)
            # obj = []
            # verb = []
            # phrase = phrase[0:1]

        if len(phrase) == 3:
            phrases.append(phrase)
            verb = []
            obj = []
            phrase = phrase[0:1]

        # print(chunk.text, "+", chunk.root.text, "+", chunk.root.dep_, "+", chunk.root.head.text)

    # if len(phrase):
    #     phrases.append(phrase)
    print("PHRASES")
    print(phrases)
    return phrases

In [122]:
# print(where_questions[10]) 
# doc = nlp(question['question']['stem'].lower())

for sent in range(0,len(where_questions)):
# for sent in range(0,20):
    doc = nlp(where_questions[sent]['question']['stem'])

    print(doc)
    print(sent)
    phrases = getSentence(doc)
    for token in doc:
        if(token.dep_ == "ROOT"):
            root = token.text
    print(root)
    print("################")

sent = where_questions[10]['question']['stem']
doc = nlp(sent.lower())
doc = nlp("If you have a empty glass and are thirsty, where would you put the glass?")
# doc = nlp("Where could you see an advertisement while reading news?")
# getParsedSentence(doc)
getSentence(doc)


see
################
Where would you see someone typing on a keyboard?
2733
PHRASES
[['someone', 'typing on', 'a keyboard']]
see
################
Where do you dip a spoon into?
2734
PHRASES
[['you', 'dip', 'a spoon']]
dip
################
Where are you if a number of athletes are wrestling?
2735
PHRASES
[['a number', 'wrestling of', 'athletes']]
are
################
Where is the mailbox between my house and the next block?
2736
PHRASES
[['the mailbox', 'is between', 'my house']]
is
################
Where would you play basketball outside?
2737
PHRASES
[['you', 'play', 'basketball']]
play
################
John's parking space was in a great position.  Where was it in relation to the building?
2738
PHRASES
[["John's parking space", 'was in', 'a great position'], ['it', 'was in', 'relation'], ['it', 'to', 'the building']]
was
################
Where could you find insects eating something green?
2739
PHRASES
[['you', 'find', 'insects'], ['you', 'eating', 'something']]
find
###############

[['you', 'have', 'a empty glass'], ['you', 'put', 'the glass']]

In [123]:
spacy.explain('pobj')

'object of preposition'

In [153]:
doc1 = nlp(where_questions[4]['question']['stem'])
doc2 = nlp(where_questions[6]['question']['stem'])
doc3 = nlp(where_questions[9]['question']['stem'])
doc4 = nlp(where_questions[12]['question']['stem'])
doc5 = nlp(where_questions[13]['question']['stem'])
doc6 = nlp(where_questions[16]['question']['stem'])
doc7 = nlp(where_questions[17]['question']['stem'])
doc8 = nlp(where_questions[18]['question']['stem'])
doc9 = nlp(where_questions[19]['question']['stem'])
doc10 = nlp(where_questions[2734]['question']['stem'])
doc11 = nlp(where_questions[2735]['question']['stem'])
doc12 = nlp(where_questions[2743]['question']['stem'])
doc13 = "If you have a empty glass and are thirsty, where would you put the glass?"


In [160]:
print(doc12)
phrases = getSentence(doc12)
# print(phrases)
print(where_questions[2743])
for token in doc12:
    if(token.dep_ == "ROOT"):
        root = token.text
print(root)

Where can you store stamps in your home?
PHRASES
[['you', 'store', 'stamps'], ['you', 'in', 'your home']]
{'answerKey': 'C', 'id': '3aebafc072d7beecf09dd37025849aaf', 'question': {'question_concept': 'stamps', 'choices': [{'label': 'A', 'text': 'table'}, {'label': 'B', 'text': 'suitcase'}, {'label': 'C', 'text': 'drawer'}, {'label': 'D', 'text': 'desk'}, {'label': 'E', 'text': 'cabinet'}], 'stem': 'Where can you store stamps in your home?'}}
store


In [161]:
import requests

def lookup(word):
    url = "http://conceptnet5.media.mit.edu/data/5.4/c/en/" + word
    obj = requests.get(url).json()
    print(obj)
    for edge in obj['edges']:
        meaning = edge['end']['@id'].split('/')
        print(meaning[-1])
        # print(edge['end']['@id'])